In [1]:
%pwd

'/home/farel/deep_learning/chicken-desease-classification/ressearch'

In [2]:
import os


os.chdir("..")
%pwd

'/home/farel/deep_learning/chicken-desease-classification'

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-11-28 14:08:21.152274: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 14:08:21.674049: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 14:08:21.674138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 14:08:21.677242: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 14:08:21.977283: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 14:08:21.981501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chicken-fecal-images/"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    
    def _valid_generator(self):
        "build a generator"
        datagenerator_kwargs = dict(rescale=1./255, validation_split=0.30)
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1],
                               batch_size=self.config.params_batch_size,
                               interpolation="bilinear")
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                       subset='validation', shuffle=False,
                                                                       **dataflow_kwargs)
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        

In [10]:
# test
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
    raise e

[2023-11-28 14:42:27,679]: INFO: common: yaml file: config/configuration.yaml loaded successfully:]
[2023-11-28 14:42:27,683]: INFO: common: yaml file: params.yaml loaded successfully:]
[2023-11-28 14:42:27,686]: INFO: common: created directory at: artifacts:]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 27s 3s/step - loss: 3.0695 - accuracy: 0.5172
[2023-11-28 14:42:55,251]: INFO: common: json file saved at: scores.json:]
